# JUAS'22 Accelerator Design Topic III: Lattice Design

The goal of this mini-workshop is to apply the knowledge gained during this school to a realistic problem. The task is to design a particle accelerator with certain specifications and boundary conditions. The idea is to gain experience how such a big task is tackled and organised.
There are ten groups of three persons each. We offer three topics, that look at the same problem from slightly different angles and with different emphasis.

**Scope: design a top factory for precision measurements.**
Design a particle collider (the *JUAS Collider*) for precision measurements of the top quark mass at the $t\bar{t}$-threshold.
The circumference must not exceed 100 km and the maximum synchrotron radiation power is limited to 50 MW per beam.
Per year at least 100000 $t\bar{t}$ pairs should be produced for sufficient statistics.

*Hint:* Should your calculation crash, it is useful to restart the notebook kernel and re-run all previous cells ("Cell $\rightarrow$ Run All Above").

In [1]:
import numpy as np

import sys, os

from cpymad.madx import Madx
from matplotlib import pyplot as plt

%matplotlib inline
import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('ticks',
              {'grid.linestyle': 'none', 'axes.edgecolor': '0',
               'axes.linewidth': 1.2, 'legend.frameon': True,
               'xtick.direction': 'out', 'ytick.direction': 'out',
               'xtick.top': True, 'ytick.right': True,
              })

In [2]:
madx = Madx(stdout=True)


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.07.00  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2021.05.03               +
  + Execution date: 2022.01.31 12:14:20      +
  ++++++++++++++++++++++++++++++++++++++++++++


*Comment:* You find the MAD-X documentation here: https://mad.web.cern.ch/mad/

## Exercise summary

1) Design basic cell according to beam requirements<br />
2) Implement a MAD-X model of your basic cell, close the ring and calculate SR integrals and equilibrium beam parameters<br />
3) Include dispersion suppressors and straight sections to your model and match optics<br />
4) Include RF cavities and calculate equilibrium beam parameters with MAD-X<br />
5) Optional: include mini-beta insertion and match optics.

### Note: this notebook embeds the structure to solve the exercises. 

$\implies$ You need to fill in the gaps indicated with "[...]" to make it work.

Also note that, at the end of the course, you are expected to present a well-prepared set of slides during the examination, suitable to show to the other groups working on different topics. You may use this notebook to compute all the results, however, the notebook itself should <b><u>not</u></b> be used in the presentation.

## 1. Assume following boundary conditions

- Beam energy: $E=175$ GeV<br />
- Horizontal equilibrium emittance: $\epsilon_x = 1.3$ nm rad<br />
- Damping partition number: $J_x =1$<br />
- A maximum circumference of 100 km<br />
- Maximum synchrotron radiation power: $P = 50$ MW per beam<br />
- The basic cell should have the same phase advance in both planes.<br />

## 2. Design of basic arc cell

### <span style="color:blue;">a) What lattice type do you choose, and why?</span>

FODO
* less expensive
* high energy for tt^bar

### <span style="color:blue;">b) What phase advance per cell do you choose, and why?</span>

In [ ]:
# phase advance per cell in radian
mu_x = [...]
mu_y = [...]

### <span style="color:blue;">c) Think  about  the  layout  of  your  basic  cell:  cell  length,  length  of  magnetic  elements,  dipole  filling factor.</span>

*Hint:*  Start with a cell length in the range from 50 m to 60 m.

In [ ]:
Lcell = [...]  # cell length in m
LB    = [...]  # length of bending magnets in m
LQ    = [...]  # length of quadrupole magnets in m
LS    = [...]  # length of sextupole magnets in m

### <span style="color:blue;">d) Calculate the bending angle for your dipoles in order to match the required equilibrium emittance.</span>

The equilibrium emittance can be approximately calculated using following equation:
$$\epsilon_x = %\frac{C_{\text{q}}}{J_x} \gamma^2\theta^3 \frac{\rho^2}{l_{\text{B}^3}} \langle\mathcal{H}\rangle = 
\frac{C_{\text{q}}}{J_x} \gamma^2\theta^3 F$$
$C_{\text{q}} = \frac{55}{32\sqrt{3}}\frac{\hbar c}{m_0c^2}=3.832\times10^{-13}$ m, $J_x\approx 1$ is the damping partition number, $\gamma$ the Lorentz factor, and $\theta$ the bending angle of all dipole magnets in a half-cell. For a FODO cell with phase advance $\mu$ the factor $F$ can be written as 
$$F_{\text{FODO}} = \frac{1}{2 \sin\mu} \frac{5+3\cos\mu}{1-\cos\mu} \frac{L}{l_{\text{B}}}.$$
$L$ is the cell length and $l_{\text{B}}$ is the length of all dipole magnets in the cell.

In [ ]:
epsilon_x = 1.3e-9 # m rad
J_x = 1
C_q = 3.832e-13 # m
gamma_L = 175/0.000551

F   = [...]
THB = [...]
THB

### <span style="color:blue;">e) Calculate the quadrupole strength $k_1$ using </span>
$$\sin(\mu/2) = \frac{L}{4f} \qquad \text{and}\qquad \frac{1}{f} = k_1 L_{\text{Q}},$$
where $L_Q$ is the length of the quadrupole magnets.

In [ ]:
k1 = [...]
k1

### <span style="color:blue;">f) Define the elements and the basic cell in MAD-X</span>

In [ ]:
inputstring = f'''
    LB = {LB};       ! length of bending magnets in m
    LQ = {LQ};       ! length of quadrupoles in m 
    LS = {LS};       ! length of sextupoles in m 
    Lcell = {Lcell}; ! length of basic cell in m
'''
madx.input(inputstring)

In [ ]:
madx.input('value, Lcell;')

In [ ]:
inputstring = f'''
    THB = {THB};

    K1QF = {k1};
    K1QD = {k1};
    
    K2SF = 0;
    K2SD = 0;
'''
madx.input(inputstring)

Definition of the elements:

In [ ]:
madx.input('''
    MB : [...];

    MQF: [...];
    MQD: [...];

    MSF: [...];
    MSD: [...];
''')

Sequence definition of arc cell:

*Hint:* Cut the first quadrupole in half (i.e. define quadrupoles with half the length) and put one at the beginning and one at the end of your sequence. This gives you a symmetric structure.

In [ ]:
madx.input('''
JC_fodo_arc : SEQUENCE, REFER=centre, L=Lcell;
    [...]
ENDSEQUENCE;
''')

Define the phase advance of the basic cell in MAD-X:

In [ ]:
# Tunes in MAD-X are given in units of 2 pi

tuneArcCell_X = [...]
tuneArcCell_Y = [...]

madx.input(f'''
    tuneArcCell_X = {tuneArcCell_X};
    tuneArcCell_Y = {tuneArcCell_Y};
''')

Set up of MAD-X environment:

In [ ]:
ENERGY = [...]
madx.input(f'ENERGY = {ENERGY};')

In [ ]:
madx.command.beam(particle='electron', energy='ENERGY')

In [ ]:
madx.use(sequence='JC_fodo_arc')

In [ ]:
madx.twiss();

### <span style="color:blue;">Check the tunes `q1` and `q2` in the TWISS summary table. Do they fit your expectation?</span>

### <span style="color:blue;">g) Match the phase advance of your basic arc cell</span>

In [ ]:
madx.input('''
match, sequence = JC_fodo_arc;
    [...]
endmatch;
''')

Check the matching result by running the twiss command:

In [ ]:
twiss=madx.twiss(centre=True)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

#### Calculate minimum and maximum beta function of your cell analytically

### <span style="color:blue;">Compare the maximum and minimum values of beta functions of MAD-X to the analytically calculated values. </span>
    
The equations are given on the lecture slides.

In [ ]:
beta_max = [...]
beta_min = [...]

beta_max, beta_min, twiss['betx'].max(), twiss['betx'].min()

Compare the maximum and minimum values of the dispersion function of MAD-X to the analytically calculated values. They are given by: $$\hat{D} = \frac{L^2}{\rho}\frac{(1+\frac{1}{2}\sin{(\mu/2)})}{4\sin^2{(\mu/2)}} \text{ and } \check{D} = \frac{L^2}{\rho}\frac{(1-\frac{1}{2}\sin{(\mu/2)})}{4\sin^2{(\mu/2)}}$$

In [ ]:
rho = LB/np.arcsin(THB)

D_max = [...]
D_min = [...]

D_max, D_min, twiss['dx'].max(), twiss['dx'].min()

### <span style="color:blue;">h) Match the chromaticity of your basic arc cell to zero.</span>

Observe the chromaticities `dq1` and `dq2` in the recent Twiss summary table. What does this mean? How can you correct this?

In [ ]:
madx.input('''
match, sequence = JC_fodo_arc;
    [...]
endmatch;
''')

In [ ]:
twiss=madx.twiss(centre=True)

Have a look at the optics again. 

### <span style="color:blue;">Can you explain why the sextupole strengths for the 'defocusing' sextupole are larger than for the 'focusing' one?</span>

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

### <span style="color:blue;">i) Build a full ring with your basic cells.</span>

### <span style="color:blue;">How many cells do you need to close the ring? </span> 

Is the overall length below the required maximum length?

In [ ]:
# Number of cells required
numberOfCells = round(np.pi*2/THB/2)
# Circumference of ring
circumference = numberOfCells*Lcell

numberOfCells, circumference

In [ ]:
madx.input('''
    numberOfCells = [...];
    L_JC_ring = numberOfCells*Lcell;
''')

Define a ring consisting of your FODO cells:

*Hint:* You can use while-loops in MAD-X sequence definitions.

In [ ]:
madx.input('''
option, -info;
i = 0;
JC_ring : SEQUENCE, refer=centre, L=L_JC_ring; 
    while (i < numberOfCells) {
        JC_fodo_arc, at=(i + 0.5) * Lcell;
        i = i + 1;
    }
ENDSEQUENCE;
option, info;
''')

In [ ]:
madx.use(sequence='JC_ring')

In [ ]:
twiss=madx.twiss()

### <span style="color:blue;">Do the tunes fit your expectation?</span>

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();
plt.xlim(0,500)

### <span style="color:blue;">Check if the ring is closed. </span>

If necessary adjust the bending angle accordingly.

In [ ]:
survey = madx.survey()

In [ ]:
survey['theta'][-1], np.pi*2

### <span style="color:blue;">j) Calculate the synchrotron radiation integrals using the 'chrom' option of the Twiss command.</span>

In [ ]:
twiss=madx.twiss(chrom=True)

You can access the results of the twiss summary table like this:

In [ ]:
twiss.summary.synch_5, twiss.summary.synch_2

### <span style="color:blue;">Calculate equilibrium emittance and energy loss per turn using the synchrotron radiation integrals. </span>

Do you obtain the emittance you were designing for?

In [ ]:
epsilon_x_SRI = [...]
U0_SRI = [...]

epsilon_x_SRI, U0_SRI

$\,$

$\,$

## 3. Dispersion suppressors and straight sections

### <span style="color:blue;">a) Design a dispersion suppressor section for both start ("DSL") and end ("DSR") of the arc sections of your storage ring and implement it in MAD-X. What scheme do you use and why? Why is it not possible to use the identical section on both sides of an arc? </span>

In [ ]:
# Define quadrupole strengths of the dispersion suppressor
madx.input('''
    K1QFDS1 := K1QF;
    K1QDDS1 := K1QD;
    K1QFDS2 := K1QF;
    K1QDDS2 := K1QD;
''')

In [ ]:
# Define elements for of the dispersion suppressor
madx.input('''
    MBDS     :  [...];

    MQFDS1   :  [...];
    MQDDS1   :  [...];
    MQFDS2   :  [...];
    MQDDS2   :  [...];
''')

Define individual sequences for your DSL and DSR.

In [ ]:
madx.input('''
LDS = 2*Lcell;
JC_dsl : SEQUENCE, REFER=centre, L=LDS;
    [...]
ENDSEQUENCE;
!
JC_dsr : SEQUENCE, REFER=centre, L=LDS;
    [...]
ENDSEQUENCE;
''')

Define a small sequence to illustrate the features of the dispersion suppressor. Use following layout: `arc cell`...`arc cell` `disp.suppr. right` `marker` `disp.suppr. left` `arc cell`...`arc cell`

In [ ]:
madx.input('''
mark_endds: marker;

match_DS : SEQUENCE, refer=centre, L = [...];
    JC_fodo_arc, at = 0.5*Lcell;
    JC_fodo_arc, at = 1.5*Lcell;
    JC_dsr, at = [...];
    mark_endds, at =[...];
    JC_dsl, at = [...];
    JC_fodo_arc, at = [...];
    JC_fodo_arc, at = [...];
ENDSEQUENCE;
''')

Save start values of periodic solutions as start values for non-periodic matching procedures:

We need to save the optics functions `betx`, `bety`, `alfx`, `alfy`, `dx`,and `dpx` as references for non-periodic Twiss calculations like for example the to illustrate how the dispersion suppressor works.

In [ ]:
madx.use(sequence='JC_fodo_arc')

In [ ]:
madx.command.savebeta(label='start.fodo.arc', place='#s')  # Comment '#s' marks the start of the sequence, '#e' the end

In [ ]:
twiss_fodo_arc=madx.twiss()

The optics functions at the beginning of your basic cell are now saved in MAD-X under the label `start.fodo.arc`. To get the beta function you can in MAD-X use: `value, start.fodo.arc->betx;` or you can save it in a variable: `foo=start.fodo.arc->betx;`

In [ ]:
madx.command.show('start.fodo.arc')

Alternatively, you can get the values from the python object `twiss_fodo_arc`, which contains the results of the twiss calculation above.

In [ ]:
twiss_fodo_arc['betx'][0],twiss_fodo_arc['bety'][0],twiss_fodo_arc['alfx'][0],twiss_fodo_arc['alfy'][0],twiss_fodo_arc['dx'][0],twiss_fodo_arc['dpx'][0]

In [ ]:
madx.use(sequence='match_DS')

Perform a non-periodic Twiss calculation with the just saved starting values and have a look at the optics:

In [ ]:
twiss=madx.twiss(betx=twiss_fodo_arc['betx'][0], 
                 bety=twiss_fodo_arc['bety'][0], 
                 alfx=twiss_fodo_arc['alfx'][0], 
                 alfy=twiss_fodo_arc['alfy'][0], 
                 dx=twiss_fodo_arc['dx'][0], 
                 dpx=twiss_fodo_arc['dpx'][0], centre=True)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

Value of the dispersion function at the end of the dispersion suppressor (center of this sequence):

In [ ]:
twiss['dx'][twiss['name'] == 'mark_endds:1'] # in m

### <span style="color:blue;">b) Define a straight cell for the straight sections </span>

In [ ]:
madx.input('''
    K1QFS := [...];
    K1QDS := [...];

    MQFS     :  [...];
    MQDS     :  [...];

JC_fodo_ss : SEQUENCE, REFER=centre, L=Lcell;
    [...]
ENDSEQUENCE;
''')

Calculate the optics of the straight cell. If necessary, re-match the optics.

In [ ]:
madx.use(sequence='JC_fodo_ss')

In [ ]:
madx.command.savebeta(label='start.fodo.ss', place='#s')

In [ ]:
twiss=madx.twiss(centre=True)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

### <span style="color:blue;">c) Define matching sections for the beginning ("MSL") and the end ("MSR") of your straight sections. Why do you need matching sections? How many parameters do you need to match? How many degrees of freedoms (=quadrupoles) do you need? </span>

In [ ]:
madx.input('''
    K1QFMS1 := K1QFS;
    K1QDMS1 := K1QDS;
    [...]
    
    MQFMS1   :  [...];
    MQDMS1   :  [...];
    [...]
''')

*Comment:* If you use quadrupoles with half the length be careful about the type of quadrupole at the beginning and at the end of the matching section. They might belong to a regular cell or the dispersion suppressor.

In [ ]:
madx.input('''
LMS=[...]; ! length of the matching section

JC_msl : SEQUENCE, REFER=centre, L=LMS;
    [...]
ENDSEQUENCE;

JC_msr : SEQUENCE, REFER=centre, L=LMS;
    [...]
ENDSEQUENCE;

''')

Define a small symmetric sequence to match the optics at the transition from the periodic solution in the arc to the one in the straight section. Use following structure: `arc cell` `DSR` `MSL` `straight cell` ... `straight cell` `MSR` `DSL` `arc cell`

In [ ]:
madx.input('''
mark_fodos_ss: marker;

match_MS : sequence, refer=centre, L = [...]*Lcell;
    [...]
    mark_endds, at = [...]; ! Marker at the end of the dispersion suppressor
    JC_msl, at = [...];
    mark_fodos_ss, at = [...]; Marker at the end of the matching section, beginning of straight FODO cell
    [...]
ENDSEQUENCE;
''')

In [ ]:
madx.use(sequence='match_MS')

Start the optics calculation with values at the begining of the arc fodo cell

In [ ]:
twiss_fodo_arc['betx'][0], twiss_fodo_arc['bety'][0], twiss_fodo_arc['alfx'][0], twiss_fodo_arc['alfy'][0],twiss_fodo_arc['dx'][0], twiss_fodo_arc['dpx'][0]

In [ ]:
twiss = madx.twiss(betx=twiss_fodo_arc['betx'][0], 
                   bety=twiss_fodo_arc['bety'][0], 
                   alfx=twiss_fodo_arc['alfx'][0], 
                   alfy=twiss_fodo_arc['alfy'][0],
                   dx=twiss_fodo_arc['dx'][0], 
                   dpx=twiss_fodo_arc['dpx'][0], centre=True)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

Match the optics. Where do you have to fulfill what constraints? Do you need to install markers in your sequence to retrieve that information?

In [ ]:
madx.input('''
match, sequence= match_MS, beta0=start.fodo.arc;
    constraint, range=mark_endds, [...];
    constraint, range=mark_fodos_ss, [...];
    vary, name=k1qfds1, step=1.0E-10;
    vary, name=k1qdds1, step=1.0E-10;
    vary, name=k1qfds2, step=1.0E-10;
    vary, name=k1qdds2, step=1.0E-10;
    vary, name=k1qfms1, step=1.0E-10;
    vary, name=k1qdms1, step=1.0E-10;
    vary, name=k1qfms2, step=1.0E-10;
    vary, name=k1qdms2, step=1.0E-10;
    vary, name=k1qdms3, step=1.0E-10;
    jacobian,calls=20,tolerance=1.e-15,bisec=3;
endmatch;
''')

In [ ]:
twiss = madx.twiss(betx=twiss_fodo_arc['betx'][0], 
                   bety=twiss_fodo_arc['bety'][0], 
                   alfx=twiss_fodo_arc['alfx'][0], 
                   alfy=twiss_fodo_arc['alfy'][0],
                   dx=twiss_fodo_arc['dx'][0], 
                   dpx=twiss_fodo_arc['dpx'][0], centre=True)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

Confirm the matching result with a periodic Twiss calculation.

In [ ]:
twiss=madx.twiss()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();

### <span style="color:blue;">d) Include straight sections at four places in your ring</span>

How do you distribute the straight sections?

In [ ]:
numberOfCells/4

In [ ]:
madx.input('''
numberOfArcCells = numberOfCells/4;
numberOfStraightCells = [...];
''')

Define sequences for your arcs, straight sections and the full ring. As this ring becomes very long, also define a sequence for one quarter of the ring to speed up matching routines (one straight section and one arc).

We use the following structure to define the arcs: <br />`dispersion suppressor left` `arc cell`...`arc cell` `dispersion suppressor right`

We use the following structure to define the straight sections: <br />`matching section left` `straight cell`...`straight cell` `matching section right`

In [ ]:
madx.input('''
Larc=numberOfArcCells*Lcell+2*LDS;

option, -info;
i = 0;
JC_arc : SEQUENCE, refer=centre, L=Larc; 
    JC_dsl, at = LDS / 2;
    while (i < numberOfArcCells) {
        JC_fodo_arc, at=LDS + (i + 0.5) * Lcell;
        i = i + 1;
    }
    JC_dsr, at=numberOfArcCells * Lcell + 1.5 * LDS;
ENDSEQUENCE;
option, info;

LSS=numberOfStraightCells*Lcell+2*LMS;

option, -info;
j=0;
JC_ss : SEQUENCE, refer=centre, L=LSS; 
    JC_msl, at = LMS/2;
    while (j<numberOfStraightCells) {
        JC_fodo_ss,  at = LMS + (j+0.5)*Lcell;
        j=j+1;
    }
    JC_msr, at =numberOfStraightCells*Lcell+1.5*LMS;
ENDSEQUENCE;
option, info;


JC_quarter : sequence, refer=centre, L=Larc+Lss;
    JC_ss, at=0.5*Lss;
    JC_arc, at = Lss+0.5*Larc;
ENDSEQUENCE;

LJC=4*(Larc+Lss);

JC : SEQUENCE, refer=centre, L=LJC;
    JC_arc, at = 0.5*Larc;
    JC_ss, at = 1.0*Larc+0.5*Lss;
    JC_arc, at = 1.5*Larc+Lss;
    JC_ss, at = 2.0*Larc+1.5*Lss;
    JC_arc, at = 2.5*Larc+2*Lss;
    JC_ss, at = 3.0*Larc+2.5*Lss;
    JC_arc, at = 3.5*Larc+3*LSS;
    JC_ss, at = 4.0*Larc+3.5*Lss;
ENDSEQUENCE;
''')

In [ ]:
madx.use(sequence='JC')

In [ ]:
twiss = madx.twiss()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();
# plt.xlim(15000,23000)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();
# plt.xlim(15000,23000)

### <span style="color:blue;">e) Observe tunes and chromaticities. Do they match your expectation? Re-match tunes and chromaticities.</span>

First, the tunes need to be matched to avoid hitting any resonances. As this storage ring is very long, the quadrupole strengths of the arc cells could be used to do the adjustment. If the phase advance of those cells is critical and should not be touched the quadrupoles of the straight cells can be used. In order to avoid distortions of the optics, an advanced matching procedure is presented in this context. <br />
We want to match following tunes:

In [ ]:
tuneToBeX = [...]
tuneToBeY = [...]

madx.input(f'''
tuneToBeX = {tuneToBeX};
tuneToBeY = {tuneToBeY};
''')

---
#### <span style="color:#af0000;">GURU mode switched on!</span>

Lean back, enjoy the show and learn:

For the matching we only use one quarter of the ring to speed up the process:

In [ ]:
madx.use(sequence='JC_quarter')

A set of markers is installed to mark the beginning of two straight cells and the beginning of the arc fodo cells. Take notice of the comments to place your markers correctly.

In [ ]:
madx.input('''
SEQEDIT, sequence=JC_quarter;
    flatten;
ENDEDIT;
SEQEDIT, sequence=JC_quarter;
    install, element=marker_s1, class=marker, at= 3*Lcell; ! start of first straight fodo cell
    install, element=marker_s2, class=marker, at= 4*Lcell; ! end of first straight fodo cell
    install, element=marker_startfodo,   class=marker, at= LSS+LDS; ! start of first arc fodo cell
    install, element=marker_startfodo2,    class=marker, at= LSS+LDS; ! end of first arc fodo cell
ENDEDIT;
SEQEDIT, sequence=JC_quarter;
    flatten;
ENDEDIT;
''')

The sequence is rearranged such that it starts with the regular arc cells and ends at the end of the dispersion suppressor AFTER the straight section. In other words, we start and end at the beginning of a regular arc cell, where we know the periodic solution of the optics functions.

In [ ]:
madx.input('''
SEQEDIT, sequence=JC_quarter;
    cycle, start=marker_startfodo;
ENDEDIT;
SEQEDIT, sequence=JC_quarter;
    flatten;
ENDEDIT;
''')

In [ ]:
madx.use(sequence='JC_quarter')

In [ ]:
madx.command.savebeta(label='start_JC', place='#S')

In [ ]:
twiss = madx.twiss()

In [ ]:
twiss['betx'][0], twiss['alfx'][0], twiss['dx'][0]

Validate the start values with a non-periodic Twiss calculation and observe the optics:

In [ ]:
madx.twiss(beta0='start_JC');

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();
plt.xlim(15000,20000)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();
plt.xlim(15000,20000)

This matching command uses expressions to tailor the constraints. Expressions allow to access values of the Twiss calculations performed during the matching run. Besides the tunes, the 3rd and 4th constraint require the same values of the beta function at the beginning and at the end of one straight cell to obtain a periodic solution in the straight section. The following constraints target the end of the sequence, where the start values need to be restored in order to obtain an overall periodic solution.<br>
Because of the large number of constraints and variables, the matching needs to be done in two steps as indicated in the MAD-X code:

In [ ]:
madx.input('''
match,use_macro;
    vary, name=K1QDDS1, step=1.0E-8;
    vary, name=K1QDDS2, step=1.0E-8;
    vary, name=K1QFDS1, step=1.0E-8;
    vary, name=K1QFDS2, step=1.0E-8;
!
    vary, name=k1qfs,   step=1.0E-8;
    vary, name=k1qds,   step=1.0E-8;
!
    vary, name=K1QDMS1, step=1.0E-8;
    vary, name=K1QDMS2, step=1.0E-8;
    vary, name=K1QDMS3, step=1.0E-8;
    vary, name=K1QFMS1, step=1.0E-8;
    vary, name=K1QFMS2, step=1.0E-8;
!
calc_twiss : macro= { USE,sequence= JC_quarter; TWISS, beta0=start_JC; };
!
    constraint, weight=10, expr=table(summ,q1)=0.25*tuneToBeX;
    constraint, weight=10, expr=table(summ,q2)=0.25*tuneToBeY;
!
    constraint, weight= 1, expr=table(twiss,marker_s2,betx)-table(twiss,marker_s1,betx)=0;
    constraint, weight= 1, expr=table(twiss,marker_s2,bety)-table(twiss,marker_s1,bety)=0;
!
    constraint, weight=10, expr=table(twiss,marker_startfodo2,betx)=start_JC->betx;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,bety)=start_JC->bety;
!
    jacobian, calls=20, tolerance=1.0E-11, bisec=3;
endmatch;
''')

In [ ]:
madx.twiss();

In [ ]:
madx.input('''
match,use_macro;
    vary, name=K1QDDS1, step=1.0E-8;
    vary, name=K1QDDS2, step=1.0E-8;
    vary, name=K1QFDS1, step=1.0E-8;
    vary, name=K1QFDS2, step=1.0E-8;
!
    vary, name=k1qfs,   step=1.0E-8;
    vary, name=k1qds,   step=1.0E-8;
!
    vary, name=K1QDMS1, step=1.0E-8;
    vary, name=K1QDMS2, step=1.0E-8;
    vary, name=K1QDMS3, step=1.0E-8;
    vary, name=K1QFMS1, step=1.0E-8;
    vary, name=K1QFMS2, step=1.0E-8;
!
calc_twiss : macro= { USE,sequence= JC_quarter; TWISS, beta0=start_JC; };
!
    constraint, weight=10, expr=table(summ,q1)=0.25*tuneToBeX;
    constraint, weight=10, expr=table(summ,q2)=0.25*tuneToBeY;
!
    constraint, weight= 1, expr=table(twiss,marker_s2,betx)-table(twiss,marker_s1,betx)=0;
    constraint, weight= 1, expr=table(twiss,marker_s2,bety)-table(twiss,marker_s1,bety)=0;
!
    constraint, weight=10, expr=table(twiss,marker_startfodo2,betx)=start_JC->betx;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,bety)=start_JC->bety;
!
    constraint, weight=10, expr=table(twiss,marker_startfodo2,alfx)=start_JC->alfx;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,alfy)=start_JC->alfy;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,dx)=start_JC->dx;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,dy)=start_JC->dy;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,dpx)=start_JC->dpx;
    constraint, weight=10, expr=table(twiss,marker_startfodo2,dpy)=start_JC->dpy;
!
    jacobian, calls=20, tolerance=1.0E-11, bisec=3;
endmatch;
''')

In [ ]:
twiss = madx.twiss();
# madx.twiss(beta0='start_JC');

In [ ]:
madx.command.value('0.25*tuneToBeX, 0.25*tuneToBeY')

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();
plt.xlim(15000,20000)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();
plt.xlim(15000,20000)

### <span style="color:#af0000;">GURU mode off! </span>

Returning to interactive mode! :-)

---

Now, reload the full storage ring and calculate tunes and chromaticity again.

In [ ]:
madx.use(sequence='JC')

In [ ]:
twiss = madx.twiss()

As a next step, we need to match the chromaticity again.

First, switch off the sextupoles and calculate the natural chromaticity of your storage ring. Can you confirm the relation $\mathcal{O}(\xi)\approx - Q$?

In [ ]:
madx.input('''
    k2sf=0;
    k2sd=0;
''')

In [ ]:
madx.twiss();

Second, rematch the chromaticities. What value do you choose and why? (Think about instabilities and collective effects.)

In [ ]:
madx.input('''
match, sequence = JC;
    global, dq1 = [...];
    global, dq2 = [...];
    vary, name=k2sf,  step=1.0E-10 ;  
    vary, name=k2sd,  step=1.0E-10 ;
    jacobian,calls=20,tolerance=1.e-15,bisec=3;
endmatch;
''')

In [ ]:
madx.twiss();

### <span style="color:blue;">f) Calculate and save the synchrotron radiation integrals of your storage ring for analytical calculations later</span>

In [ ]:
twiss = madx.twiss(chrom=True)

In [ ]:
SRI1 = twiss.summary.synch_1
SRI2 = twiss.summary.synch_2
SRI3 = twiss.summary.synch_3
SRI4 = twiss.summary.synch_4
SRI5 = twiss.summary.synch_5

$\,$

$\,$

## 4. RF sections

## <span style="color:#af0000;">Let it shine! ;-)</span>

Before including radiation effects, RF cavities need to be defined and installed in the lattice.
In this exercise we use an RF system designed for FCC-ee. It uses 400 MHz cavities that consist of four cells with a length of 0.375 m each.
What is the overall voltage you assume to guarantee stable operation?

The energy gain in the cavity is given by 

$$U = eV_{RF} \sin{(2\pi (\phi-hf_0t))},$$

where $\phi$ is the phase lag ("synchronous phase"), $h$ the harmonic number of the ring, and  $f_0$ the cavity frequency.

### a) Estimate the total RF voltage which is needed to compensate the synchrotron radiation energy loss.

In [ ]:
VRF = [...]

madx.input(f'''
    switch_RF_on = 0;    ! this switch allows to set the cavity voltage to Zero
    VRF  = {VRF};        ! overall voltage of all cavities
    LGRF = 0.5;          ! phase lag 
    numberOfCav400 = 160;

    ! 400 MHz system
    LCAVcell400         = 0.375;
    harmonic400         = 114746;
    numberOfCellsCav400 = 4;
    Vcell400           := VRF / numberOfCav400 / numberOfCellsCav400 * switch_RF_on;                ! = [...] MV
    LGRF400            := cos(switch_RF_on * pi / 2) / 2 + sin(switch_RF_on * pi / 2) * LGRF;       ! = 0.5, if RF switched off, and LGRF, if RF switched on

    CAVcell400: RFCAVITY, L:=LCAVcell400, VOLT:=Vcell400, HARMON:=harmonic400, LAG:=LGRF400;

CavModule : SEQUENCE, refer=centre, L=numberOfCellsCav400 + 0.5 * (numberOfCellsCav400 - 1) * LCAVcell400;
    CAVcell400, at=0.5 * LCAVcell400;
    CAVcell400, at=2.0 * LCAVcell400;
    CAVcell400, at=3.5 * LCAVcell400;
    CAVcell400, at=5.0 * LCAVcell400;
ENDSEQUENCE;
''')

### <span style="color:blue;">b) Define a straight cell and straight sections that contain RF cavities (please check if the definition is consistent to yours):</span>

In [ ]:
madx.input('''
JC_fodo_ss_RF : SEQUENCE, REFER=centre, L=Lcell;
    MQFS, at=0.5*LQ/2;
    CavModule, at = 0.25*Lcell;
    MQDS, at=0.5*Lcell-0.5*LQ/2;
    MQDS, at=0.5*Lcell+0.5*LQ/2;
    CavModule, at=0.75*Lcell;
    MQFS, at=Lcell-0.5*LQ/2;
ENDSEQUENCE;
''')

Define a straight section and the full collider lattice as above but including RF cavities:

In [ ]:
madx.input('''
option, -info;
j=0;
JC_ss_RF : SEQUENCE, refer=centre, L=LSS; 
    JC_msl, at = LMS/2;
    while (j<numberOfStraightCells)	{
        JC_fodo_ss_RF,  at = LMS + (j+0.5)*Lcell;
        j=j+1;
    }
    JC_msr, at =numberOfStraightCells*Lcell+1.5*LMS;
ENDSEQUENCE;
option, info;

JC_quarter_RF : sequence, refer=centre, L=Larc+Lss;
    JC_ss_RF, at=0.5*Lss;
    JC_arc, at = Lss+0.5*Larc;
ENDSEQUENCE;

JC_RF : SEQUENCE, refer=centre, L=LJC;
    JC_arc, at = 0.5*Larc;
    JC_ss_RF, at = 1.0*Larc+0.5*Lss;
    JC_arc, at = 1.5*Larc+Lss;
    JC_ss_RF, at = 2.0*Larc+1.5*Lss;
    JC_arc, at = 2.5*Larc+2*Lss;
    JC_ss_RF, at = 3.0*Larc+2.5*Lss;
    JC_arc, at = 3.5*Larc+3*LSS;
    JC_ss_RF, at = 4.0*Larc+3.5*Lss;
ENDSEQUENCE;
''')

### <span style="color:blue;">c) Switch on radiation and observe tunes and chromaticities</span>

In [ ]:
madx.command.beam(radiate=True)

Switch on RF system. Start with w phase lag of 0.5, otherwise calculate the required value you need from the equation for the energy gain.

In [ ]:
madx.input('''
    switch_RF_on = 1;
    lgrf = 0.5;
''')

Try a first Twiss calculation including radiation effects:

In [ ]:
madx.use(sequence='JC_RF')

In [ ]:
madx.twiss();

What happened to tunes, chromaticities and horizontal orbit? Can you explain?

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();
# plt.xlim(15000,20000)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();
# plt.xlim(65000,75000)

First we have to match the energy gain in our cavities to make sure we receive as much energy as we lose due to synchrotron radiation:

In [ ]:
madx.input('''
match, sequence=JC_RF;
    vary, name = lgrf, step=1.0E-6;
    constraint, sequence=JC_RF, range = #E, T=0;
    jacobian, calls=30, tolerance=1.E-22, strategy=3;
endmatch;
''')

In [ ]:
madx.twiss();

As a next step we rematch tunes and chromaticities. Since we have a big beta beat, we do not use the complicated matching command from above.

In [ ]:
madx.input('''
match, sequence = JC_RF;
    global, q1=tuneToBeX;
    global, q2=tuneToBeY;
    global, dq1=2;
    global, dq2=2;
    vary, name=k1qfs,  step=1.0E-10 ;  
    vary, name=k1qds,  step=1.0E-10 ;
    vary, name=k2sf,  step=1.0E-10 ;  
    vary, name=k2sd,  step=1.0E-10 ;
    jacobian,calls=20,tolerance=1.e-15,bisec=3;
endmatch;
''')

In [ ]:
twiss = madx.twiss()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['betx'], label=r'$\beta_x$', color='black')
plt.plot(twiss['s'], twiss['bety'], label=r'$\beta_y$', color='red')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Beta function $\beta_{x,y}$ [m]')
plt.legend();
# plt.xlim(15000,20000)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['Dx'], label=r'$D_x$', color='green')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Dispersion function $D_x$ [m]')
plt.legend();
# plt.xlim(65000,75000)

### <span style="color:blue;">d) Observe the transverse orbit. Can you explain the pattern you see?</span>

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(twiss['s'], twiss['x'], label=r'$x$', color='black')
plt.xlabel('Path length $s$ [m]')
plt.ylabel(r'Horizontal orbit $x$ [m]')
plt.legend();
# plt.xlim(15000,20000)

This effect has already been observed in LEP measurements at the time:

![sawtooth orbit](sawtooth.png)

### <span style="color:blue;">e) Calculate the equilibrium beam parameters with the MAD-X emit command</span>

In [ ]:
madx.emit();

Compare the results from emit with the anaytically calculated values:

In [ ]:
epsilon_x_SRI = C_q*gamma_L**2/J_x*SRI5/SRI2
U0_SRI = 8.846e-5/2/np.pi*175**4*SRI2 # in GeV
alpha_c_SRI = SRI1/twiss.summary.length
dp_p_SRI = np.sqrt(C_q * gamma_L**2 * SRI3 / (2 * SRI2 + SRI4))
J_x_SRI = 1-SRI4/SRI2
J_s_SRI = 2+SRI4/SRI2

epsilon_x_SRI, U0_SRI, alpha_c_SRI,dp_p_SRI,J_x_SRI,J_s_SRI

### <span style="color:blue;">f) How many particles and bunches can you fill into the ring before you reach the limit of synchrotron radiation power?</span>

$\,$

$\,$

## 5. If time permits: mini-beta insertions


### <span style="color:blue;">a) Calculate the required values for $\beta^*$ or get them from colleagues (groups working on the general parameters topic I).</span>

### <span style="color:blue;">b) Include two mini-beta insertions into your MAD-X model and match the optics.</span>

### <span style="color:blue;">c) How do you distribute the mini-beta insertions and why?</span>